# Notebook 4 - Harmonisation robuste (MAE)
Pipeline pour tester l'harmonisation avec toutes les méthodes robustes et calculer le MAE.

In [13]:
import os
import pandas as pd
from joblib import Parallel, delayed

from robust_evaluation_tools.robust_utils import get_site, get_camcan_file, get_metrics, remove_covariates_effects_metrics
from robust_evaluation_tools.robust_harmonization import fit, apply, compare_with_compilation, compare_with_compilation_SMAPE, compare_with_compilation_STD, compare_with_compilation_var
from robust_evaluation_tools.robust_outlier_detection import z_score_detection, flag_sid
from robust_evaluation_tools.robust_MLP import predict_malades_MLP


In [ ]:
# Parametres
MAINFOLDER = "RESULTS/MAE_THRESHOLDS"
base_name = "harmonized"
augmentation_copies = 5
augmentation_suffix = f"_AUG_{augmentation_copies}" if augmentation_copies else ""
SYNTHETIC_SITES = f"DONNES/processed/synthetic_sites/{base_name}{augmentation_suffix}"
SYNTHETIC_SITES_VERSION = "v1"

SMAPE_ONLY = False
harmonization_method = "classic"

metrics = get_metrics()
diseases = ["ALL"]

BASE_METHODS = ["No", "raw"]
ROBUST_METHODS = [
    "IQR",
    "MAD",
    "MMS",
    "G_ZS",
    "ZS",
    "SN",
    "QN",
    "MLP7_ALL"
]

# Seuils a tester par methode (None => seuil par defaut defini dans clinical_combat.utils.robust)
ROBUST_THRESHOLDS = {
    "IQR": [1.0, 1.25, 1.5, 1.75, 2.0],
    "MAD": [2.5, 3, 3.5, 4, 4.5],
    "MMS": [0.0005, 0.001, 0.002],
    "G_ZS": [0.8, 1.0, 1.2, 1.5],
    "ZS": [2, 2.5, 3, 3.5, 4],
    "SN": [2, 2.5, 3, 3.5, 4],
    "QN": [2, 2.5, 3, 3.5, 4],
    "MLP7_ALL": [0.5, 0.6, 0.7, 0.8, 0.9]
}

def format_threshold_label(threshold):
    if threshold is None:
        return "default"
    return str(threshold).replace(".", "p")

def expand_robust_methods(base_methods, robust_methods, thresholds_map):
    configs = [{"method": m, "label": m, "threshold": None} for m in base_methods]
    for method in robust_methods:
        thresholds = thresholds_map.get(method, [None])
        for th in thresholds:
            label_suffix = "" if th is None else f"_thr{format_threshold_label(th)}"
            configs.append({"method": method, "label": f"{method}{label_suffix}", "threshold": th})
    return configs

ALL_METHODS = expand_robust_methods(BASE_METHODS, ROBUST_METHODS, ROBUST_THRESHOLDS)

# sample_sizes = [100]
# disease_ratios = [0.03, 0.1, 0.3, 0.5, 0.7, 0.8]
# num_tests = 20
n_jobs_number = -1

sample_sizes = None
disease_ratios = None
num_tests = None


## Fonctions d'harmonisation


In [15]:
def harmonize(f_train, ref_data_file, metric, harmonizartion_method, f_test, directory, method, robust_config, rwp, hc, gt_train_file_name, gt_test_file_name):
    robust_method = robust_config.get('method')
    robust_label = robust_config.get('label', robust_method)
    robust_threshold = robust_config.get('threshold')

    dir_path = os.path.join(directory, robust_label) if method == 'robust' else os.path.join(directory, method)

    if robust_label == 'raw':
        output_filename_train = f_train
        output_filename_test = f_test
    else:
        output_model_filename = fit(f_train, ref_data_file, metric, harmonizartion_method, robust_method, rwp, dir_path, hc, robust_label=robust_label, robust_threshold=robust_threshold)
        output_filename_train = apply(f_train, output_model_filename, metric, harmonizartion_method, robust_method, rwp, dir_path, robust_label=robust_label)
        output_filename_test = apply(f_test, output_model_filename, metric, harmonizartion_method, robust_method, rwp, dir_path, robust_label=robust_label)

    out_train_df = pd.read_csv(output_filename_train)
    out_test_df = pd.read_csv(output_filename_test)
    gt_train_df = pd.read_csv(gt_train_file_name)
    gt_test_df = pd.read_csv(gt_test_file_name)

    std_mae_train = compare_with_compilation_STD(out_train_df, gt_train_df)
    std_mae_test = compare_with_compilation_STD(out_test_df, gt_test_df)

    if not SMAPE_ONLY:
        smape_train = compare_with_compilation_SMAPE(out_train_df, gt_train_df)
        smape_test = compare_with_compilation_SMAPE(out_test_df, gt_test_df)

        mae_train = compare_with_compilation(out_train_df, gt_train_df)
        maev_train = compare_with_compilation_var(out_train_df, gt_train_df)

        mae_test = compare_with_compilation(out_test_df, gt_test_df)
        maev_test = compare_with_compilation_var(out_test_df, gt_test_df)
    else:
        mae_test = pd.DataFrame()
        maev_test = pd.DataFrame()
        mae_train = pd.DataFrame()
        maev_train = pd.DataFrame()
        smape_test = pd.DataFrame()
        smape_train = pd.DataFrame()

    meta_cols = {'site': get_site(f_train), 'method': method, 'robust_method': robust_label, 'robust_threshold': robust_threshold}
    for df in (mae_test, maev_test, mae_train, maev_train, smape_test, smape_train, std_mae_test, std_mae_train):
        for k, v in meta_cols.items():
            df[k] = v

    return mae_test, maev_test, mae_train, maev_train, smape_test, smape_train, std_mae_test, std_mae_train


def analyse_site(f_train, f_test, robust_methods, directory, ref_data_file, metric, harmonizartion_method, gt_train_file_name, gt_test_file_name):
    df_mae_test_robust = pd.DataFrame()
    df_maev_test_robust = pd.DataFrame()
    df_mae_train_robust = pd.DataFrame()
    df_maev_train_robust = pd.DataFrame()
    df_smape_test_robust = pd.DataFrame()
    df_smape_train_robust = pd.DataFrame()
    df_std_mae_test_robust = pd.DataFrame()
    df_std_mae_train_robust = pd.DataFrame()

    for robust in robust_methods:
        label = robust.get('label')
        if label == 'raw':
            (mae_test, maev_test, mae_train, maev_train, smape_test, smape_train, std_mae_test, std_mae_train) = harmonize(
                f_train, ref_data_file, metric, harmonizartion_method, f_test, directory, 'raw', robust, False, False, gt_train_file_name, gt_test_file_name
            )
        elif label == 'No':
            (mae_test_hc, maev_test_hc, mae_train_hc, maev_train_hc, smape_test_hc, smape_train_hc, std_mae_test_hc, std_mae_train_hc) = harmonize(
                f_train, ref_data_file, metric, harmonizartion_method, f_test, directory, 'hc', robust, False, True, gt_train_file_name, gt_test_file_name
            )
            (mae_test_no, maev_test_no, mae_train_no, maev_train_no, smape_test_no, smape_train_no, std_mae_test_no, std_mae_train_no) = harmonize(
                f_train, ref_data_file, metric, harmonizartion_method, f_test, directory, 'NoRobust', robust, False, False, gt_train_file_name, gt_test_file_name
            )

            mae_test = pd.concat([mae_test_no, mae_test_hc], ignore_index=True)
            maev_test = pd.concat([maev_test_no, maev_test_hc], ignore_index=True)
            mae_train = pd.concat([mae_train_no, mae_train_hc], ignore_index=True)
            maev_train = pd.concat([maev_train_no, maev_train_hc], ignore_index=True)
            smape_test = pd.concat([smape_test_no, smape_test_hc], ignore_index=True)
            smape_train = pd.concat([smape_train_no, smape_train_hc], ignore_index=True)
            std_mae_test = pd.concat([std_mae_test_no, std_mae_test_hc], ignore_index=True)
            std_mae_train = pd.concat([std_mae_train_no, std_mae_train_hc], ignore_index=True)
        else:
            (mae_test, maev_test, mae_train, maev_train, smape_test, smape_train, std_mae_test, std_mae_train) = harmonize(
                f_train, ref_data_file, metric, harmonizartion_method, f_test, directory, 'robust', robust, False, False, gt_train_file_name, gt_test_file_name
            )

        df_mae_test_robust = pd.concat([df_mae_test_robust, mae_test], ignore_index=True)
        df_maev_test_robust = pd.concat([df_maev_test_robust, maev_test], ignore_index=True)
        df_mae_train_robust = pd.concat([df_mae_train_robust, mae_train], ignore_index=True)
        df_maev_train_robust = pd.concat([df_maev_train_robust, maev_train], ignore_index=True)
        df_smape_test_robust = pd.concat([df_smape_test_robust, smape_test], ignore_index=True)
        df_smape_train_robust = pd.concat([df_smape_train_robust, smape_train], ignore_index=True)
        df_std_mae_test_robust = pd.concat([df_std_mae_test_robust, std_mae_test], ignore_index=True)
        df_std_mae_train_robust = pd.concat([df_std_mae_train_robust, std_mae_train], ignore_index=True)

    return (df_mae_test_robust, df_maev_test_robust, df_mae_train_robust, df_maev_train_robust, df_smape_test_robust, df_smape_train_robust, df_std_mae_test_robust, df_std_mae_train_robust)


def process_analysis(disease, sample_size, disease_ratio, test_index, harmonization_method, SYNTHETIC_SITES_VERSION, metrics, robust_methods):
    sizeDir = os.path.join(MAINFOLDER, f'PROCESS_{harmonization_method}', disease, f"{sample_size}_{int(disease_ratio * 100)}", f"{test_index}")
    mae_test_file_path = os.path.join(sizeDir, 'mae_compilation_test.csv')
    maev_test_file_path = os.path.join(sizeDir, 'maev_compilation_test.csv')
    mae_train_file_path = os.path.join(sizeDir, 'mae_compilation_train.csv')
    maev_train_file_path = os.path.join(sizeDir, 'maev_compilation_train.csv')
    smape_test_file_path = os.path.join(sizeDir, 'smape_compilation_test.csv')
    smape_train_file_path = os.path.join(sizeDir, 'smape_compilation_train.csv')
    std_mae_train_file_path = os.path.join(sizeDir, 'std_mae_compilation_train.csv')
    std_mae_test_file_path = os.path.join(sizeDir, 'std_mae_compilation_test.csv')

    def _load_if_exists(path):
        return pd.read_csv(path) if os.path.isfile(path) else pd.DataFrame()

    mae_compilation_test = _load_if_exists(mae_test_file_path)
    maev_compilation_test = _load_if_exists(maev_test_file_path)
    mae_compilation_train = _load_if_exists(mae_train_file_path)
    maev_compilation_train = _load_if_exists(maev_train_file_path)
    smape_compilation_test = _load_if_exists(smape_test_file_path)
    smape_compilation_train = _load_if_exists(smape_train_file_path)
    std_mae_compilation_train = _load_if_exists(std_mae_train_file_path)
    std_mae_compilation_test = _load_if_exists(std_mae_test_file_path)

    methods_to_run = robust_methods
    if not std_mae_compilation_test.empty:
        existing_methods = set(std_mae_compilation_test['robust_method'].unique())
        missing_methods = [m for m in methods_to_run if m.get('label') not in existing_methods]
        if not missing_methods:
            print(f"✔️ Toutes les methodes deja traitees pour {disease} {sample_size}_{int(disease_ratio*100)} test_index {test_index}.")
            return (mae_test_file_path, maev_test_file_path, mae_train_file_path, maev_train_file_path, smape_test_file_path, smape_train_file_path, std_mae_test_file_path, std_mae_train_file_path)
        methods_to_run = missing_methods

    directory_site = os.path.join(SYNTHETIC_SITES, disease)
    sizeDir_site = os.path.join(directory_site, f"{sample_size}_{int(disease_ratio * 100)}")
    tempDir_site = os.path.join(sizeDir_site, f"{test_index}")

    df_for_tags = pd.read_csv(os.path.join(tempDir_site, f"train_{sample_size}_{int(disease_ratio * 100)}_{test_index}_all.csv"))
    df_for_tags = df_for_tags[~df_for_tags['bundle'].isin(['left_ventricle', 'right_ventricle'])]
    df_for_tags = remove_covariates_effects_metrics(df_for_tags)

    z_score_sids = z_score_detection(df_for_tags)

    models = ['mlp7_ALL']
    preds = {m.upper(): predict_malades_MLP(df_for_tags, m) for m in models}

    for metric in metrics:
        tempDir = os.path.join(sizeDir, metric)
        os.makedirs(tempDir, exist_ok=True)

        train_file_name = f"train_{sample_size}_{int(disease_ratio * 100)}_{test_index}_{metric}.csv"
        test_file_name = f"test_{sample_size}_{int(disease_ratio * 100)}_{test_index}_{metric}.csv"
        gt_train_file_name = os.path.join(tempDir_site, f"gt_train_{sample_size}_{int(disease_ratio * 100)}_{test_index}_{metric}.csv")
        gt_test_file_name = os.path.join(tempDir_site, f"gt_test_{sample_size}_{int(disease_ratio * 100)}_{test_index}_{metric}.csv")

        train_file = os.path.join(tempDir_site, train_file_name)
        train_df = pd.read_csv(train_file)
        train_df = train_df[~train_df['bundle'].isin(['left_ventricle', 'right_ventricle'])]
        train_df = train_df.drop(columns=['mean_no_cov', 'metric_bundle'], errors='ignore')
        train_df = flag_sid(train_df, z_score_sids, 'G_ZS')
        for label, sid_list in preds.items():
            train_df = flag_sid(train_df, sid_list, label)

        train_df['site'] = disease + '_' + train_df['site']
        new_train_file = os.path.join(tempDir, train_file_name)
        train_df.to_csv(new_train_file, index=False)

        test_file = os.path.join(tempDir_site, test_file_name)
        test_df = pd.read_csv(test_file)
        test_df = test_df[~test_df['bundle'].isin(['left_ventricle', 'right_ventricle'])]
        test_df = test_df.drop(columns=['mean_no_cov', 'metric_bundle'], errors='ignore')
        test_df['site'] = train_df['site']
        new_test_file = os.path.join(tempDir, test_file_name)
        test_df.to_csv(new_test_file, index=False)

        ref_data_file = get_camcan_file(metric, cleaned=True)

        mae_analyze_test, maev_analyze_test, mae_analyze_train, maev_analyze_train, smape_analyze_test, smape_analyze_train, std_mae_analyze_test, std_mae_analyze_train = analyse_site(
            new_train_file, new_test_file, methods_to_run, tempDir, ref_data_file, metric, harmonization_method, gt_train_file_name, gt_test_file_name
        )

        for df in [mae_analyze_test, maev_analyze_test, mae_analyze_train, maev_analyze_train, smape_analyze_test, smape_analyze_train, std_mae_analyze_test, std_mae_analyze_train]:
            df['disease'] = disease
            df['metric'] = metric

        std_mae_compilation_test = pd.concat([std_mae_compilation_test, std_mae_analyze_test], ignore_index=True).drop_duplicates()
        std_mae_compilation_train = pd.concat([std_mae_compilation_train, std_mae_analyze_train], ignore_index=True).drop_duplicates()
        if not SMAPE_ONLY:
            mae_compilation_test = pd.concat([mae_compilation_test, mae_analyze_test], ignore_index=True).drop_duplicates()
            maev_compilation_test = pd.concat([maev_compilation_test, maev_analyze_test], ignore_index=True).drop_duplicates()
            mae_compilation_train = pd.concat([mae_compilation_train, mae_analyze_train], ignore_index=True).drop_duplicates()
            maev_compilation_train = pd.concat([maev_compilation_train, maev_analyze_train], ignore_index=True).drop_duplicates()
            smape_compilation_test = pd.concat([smape_compilation_test, smape_analyze_test], ignore_index=True).drop_duplicates()
            smape_compilation_train = pd.concat([smape_compilation_train, smape_analyze_train], ignore_index=True).drop_duplicates()

    os.makedirs(sizeDir, exist_ok=True)
    std_mae_compilation_test.to_csv(std_mae_test_file_path, index=False)
    std_mae_compilation_train.to_csv(std_mae_train_file_path, index=False)
    if not SMAPE_ONLY:
        mae_compilation_test.to_csv(mae_test_file_path, index=False)
        maev_compilation_test.to_csv(maev_test_file_path, index=False)
        mae_compilation_train.to_csv(mae_train_file_path, index=False)
        maev_compilation_train.to_csv(maev_train_file_path, index=False)
        smape_compilation_test.to_csv(smape_test_file_path, index=False)
        smape_compilation_train.to_csv(smape_train_file_path, index=False)



def infer_experiment_settings(base_dir, diseases):
    sample_sizes = set()
    disease_ratios = set()
    inferred_num_tests = 0

    for disease in diseases:
        disease_dir = os.path.join(base_dir, disease)
        if not os.path.isdir(disease_dir):
            continue

        for size_ratio_name in os.listdir(disease_dir):
            size_ratio_path = os.path.join(disease_dir, size_ratio_name)
            if not os.path.isdir(size_ratio_path) or '_' not in size_ratio_name:
                continue

            size_part, ratio_part = size_ratio_name.split('_', 1)
            try:
                sample_size_val = int(size_part)
                ratio_val = int(ratio_part) / 100
            except ValueError:
                continue

            sample_sizes.add(sample_size_val)
            disease_ratios.add(ratio_val)

            possible_tests = [
                int(p)
                for p in os.listdir(size_ratio_path)
                if p.isdigit() and os.path.isdir(os.path.join(size_ratio_path, p))
            ]
            if possible_tests:
                inferred_num_tests = max(inferred_num_tests, max(possible_tests) + 1, len(possible_tests))

    if not sample_sizes or not disease_ratios or inferred_num_tests == 0:
        raise ValueError(f"Impossible de deduire les parametres depuis {base_dir} pour les maladies {diseases}.")

    return sorted(sample_sizes), sorted(disease_ratios), inferred_num_tests

def analyse_method(sample_sizes, disease_ratios, num_tests, robust_methods, diseases, metrics, harmonization_method, SYNTHETIC_SITES_VERSION, n_jobs=-1):
    if sample_sizes is None or disease_ratios is None or num_tests is None:
        inferred_sample_sizes, inferred_disease_ratios, inferred_num_tests = infer_experiment_settings(SYNTHETIC_SITES, diseases)
        sample_sizes = inferred_sample_sizes if sample_sizes is None else sample_sizes
        disease_ratios = inferred_disease_ratios if disease_ratios is None else disease_ratios
        num_tests = inferred_num_tests if num_tests is None else num_tests

    tasks = [
        (disease, sample_size, disease_ratio, num_test, harmonization_method, SYNTHETIC_SITES_VERSION, metrics, robust_methods)
        for disease in diseases
        for sample_size in sample_sizes
        for disease_ratio in disease_ratios
        for num_test in range(num_tests)
    ]

    Parallel(n_jobs=n_jobs)(delayed(process_analysis)(*task) for task in tasks)


## Lancement de l'analyse


In [16]:
analyse_method(sample_sizes, disease_ratios, num_tests, ALL_METHODS, diseases, metrics, harmonization_method, SYNTHETIC_SITES_VERSION, n_jobs=n_jobs_number)


Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with method MLP7_ALL
Removing outliers with me